In [ ]:
import pandas as pd
from difflib import SequenceMatcher
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
import pandas as pd

In [ ]:
# 데이터 불러오기
apt_df = pd.read_csv('./아파트_리스트.csv')
apart_df = pd.read_csv('./실거래가_데이터.csv')


In [ ]:
# 문제 데이터 갯수 확인
len(apt_df.loc[(apt_df['road']!=apt_df['new_road'])].index)

In [ ]:
# 빈 리스트 정의
apt_lst2 = []

In [ ]:
# 전처리 함수
for a1 in range(0, len(apt_df.loc[(apt_df['road']!=apt_df['new_road'])].index)):
    a = apt_df.loc[(apt_df['road']!=apt_df['new_road'])].index[a1]
    apt = apt_df.apt[a]
    road = apt_df.road[a]
    sch = (apart_df[(apart_df.동_단지.str.startswith(apt_df.apt[a])) & (apart_df.도로명 == apt_df.road[a])]['시군구'] + ' ' + apart_df[(apart_df.동_단지.str.startswith(apt_df.apt[a])) & (apart_df.도로명 == apt_df.road[a])]['번지']).unique()[0]
    sch_road = apart_df[(apart_df.동_단지.str.startswith(apt_df.apt[a])) & (apart_df.도로명 == apt_df.road[a])]['시군구'].unique()[0].split(' ')[-2] + ' ' + road
    zibun = sch.split('특별시 ')[-1]
    apt_nm = 'None'
    road_nm = 'None'
    lat = '0'
    long = '0'
    home = '0'
    apt_company = 'None'
    size = '0'
    room = '0'
    bath = '0'
    bigo = 'None'
    zibun_nm = 'None'
    rt = 0
    max_i = 0
    web = Chrome('../chromedriver.exe')
    web.get('https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=' + sch)
    web.implicitly_wait(5)
    
    try:
        web.find_element(By.CSS_SELECTOR, '#loc-main-section-root > section > div > div._3ttP1 > div > div._2p8Of > div._2ZgN5 > div._1oJkn > span:nth-child(2)')
        apt_nm = web.find_element(By.CSS_SELECTOR, '#loc-main-section-root > section > div > div._3ttP1 > div > div._2p8Of > div._2ZgN5 > div._1oJkn > span:nth-child(2)').text.split('도로명')[0]
    except:
        web.get('https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=' + sch_road)
        web.implicitly_wait(5)
        try:
            web.find_element(By.CSS_SELECTOR, '#loc-main-section-root > section > div > div._3ttP1 > div > div._2p8Of > div._2ZgN5 > div._1oJkn > span:nth-child(2)')
            apt_nm = web.find_element(By.CSS_SELECTOR, '#loc-main-section-root > section > div > div._3ttP1 > div > div._2p8Of > div._2ZgN5 > div._1oJkn > span:nth-child(2)').text.split('지번')[0]
        except:
            apt_nm=''


    if apt_nm != '':
        web.get('https://new.land.naver.com/complexes/667?ms=37.483998,127.056011,17&a=APT:ABYG:JGC&e=RETAIL')
        web.implicitly_wait(5)
        web.find_element(By.ID, 'search_input').clear()
        web.find_element(By.ID, 'search_input').send_keys(apt_nm)
        web.find_element(By.XPATH,'//*[@id="wrap"]/header/div[1]/form/fieldset/button[1]').click()
        web.implicitly_wait(5)

        try:
            web.find_element(By.CSS_SELECTOR,'#ct > div.map_wrap > div.search_panel > div.list_fixed > div')
            try:
                web.find_element(By.XPATH, '//*[@id="ct"]/div[2]/div[2]/div[2]/div/div/div/div/i')
                print('error', a1,  a, apt)
                apt_lst2.append({'apt':apt,'zibun':zibun, 'new_zibun':zibun_nm, 'new_apt':apt_nm, 'road':road, 'new_road':road_nm, 'latitude':lat, 'longitude':long, 'company':apt_company, 'home_cnt':home, 'home_size':size, 'room_cnt':room, 'bath_cnt':bath, 'bigo':bigo})
            except:
                num_lst = []
                for i in range(len(web.find_element(By.CSS_SELECTOR, '#ct > div.map_wrap > div.search_panel > div.list_contents > div').find_elements(By.CLASS_NAME, 'item'))):
                    if (len(web.find_element(By.CSS_SELECTOR,'#ct > div.map_wrap > div.search_panel > div.list_contents > div > div > div:nth-child('+str(i+1)+') > div > a').find_elements(By.CSS_SELECTOR,'*')) > 1):
                        if (web.find_element(By.CSS_SELECTOR, '#ct > div.map_wrap > div.search_panel > div.list_contents > div > div > div:nth-child('+str(i+1)+') > div > a > div.info_area > strong').text == '아파트') & (web.find_element(By.CSS_SELECTOR, '#ct > div.map_wrap > div.search_panel > div.list_contents > div > div > div:nth-child('+str(i+1)+') > div > a > div.address').text.endswith(apart_df[(apart_df.동_단지.str.startswith(apt_df.apt[a])) & (apart_df.도로명 == apt_df.road[a])]['시군구'].unique()[0].split('특별시 ')[-1])):
                            num_lst.append(i)
                if len(num_lst)==1:
                    max_i = num_lst[0]
                elif len(num_lst) > 1:
                    for ii in num_lst:
                        web.find_element(By.CSS_SELECTOR, '#ct > div.map_wrap > div.search_panel > div.list_contents > div > div > div:nth-child('+str(ii+1)+')').click()
                        web.implicitly_wait(5)
                        web.find_element(By.XPATH,'//*[@id="summaryInfo"]/div[2]/div[2]/button[1]').click()
                        web.implicitly_wait(5)
                        if (web.find_elements(By.CSS_SELECTOR,'#detailContents1 > div.detail_box--complex > table > tbody > tr:nth-child(7) > td > p:nth-child(2)')[-1].text.split('구 ')[-1].split(' ')[0].endswith(road.split(' ')[0])) or (web.find_element(By.CSS_SELECTOR, '#detailContents1 > div.detail_box--complex > table > tbody > tr:nth-child(7) > td > p:nth-child(1)').text.split('-')[0].endswith(sch.split('특별시 ')[-1].split('-')[0])):
                            zibun_nm1 = web.find_element(By.CSS_SELECTOR, '#detailContents1 > div.detail_box--complex > table > tbody > tr:nth-child(7) > td > p:nth-child(1)').text.split('시 ')[-1]
                            road_nm1 = web.find_elements(By.CLASS_NAME,'address')[-1].text.split('구 ')[-1]
                            if max(SequenceMatcher(None,zibun,zibun_nm1).ratio(), SequenceMatcher(None,road,road_nm1).ratio()) > rt:
                                rt = max(SequenceMatcher(None,zibun,zibun_nm1).ratio(), SequenceMatcher(None,road,road_nm1).ratio())
                                max_i = ii
                        web.find_element(By.ID, 'search_input').send_keys(apt_nm)
                        web.find_element(By.XPATH,'//*[@id="wrap"]/header/div[1]/form/fieldset/button[1]').click()
                        web.implicitly_wait(5)
                web.find_element(By.CSS_SELECTOR, '#ct > div.map_wrap > div.search_panel > div.list_contents > div > div > div:nth-child('+str(max_i+1)+')').click()
                web.implicitly_wait(5)
                web.find_element(By.XPATH,'//*[@id="summaryInfo"]/div[2]/div[2]/button[1]').click()
                web.implicitly_wait(5)
                zibun_nm = web.find_element(By.CSS_SELECTOR, '#detailContents1 > div.detail_box--complex > table > tbody > tr:nth-child(7) > td > p:nth-child(1)').text.split('시 ')[-1]
                road_nm = web.find_elements(By.CLASS_NAME,'address')[-1].text.split('구 ')[-1]
                apt_nm = web.find_element(By.ID,'complexTitle').text
                lat = web.current_url.split('=')[1].split(',')[0]
                long = web.current_url.split('=')[1].split(',')[1]
                home = web.find_element(By.CSS_SELECTOR,'#detailContents1 > div.detail_box--complex > table > tbody > tr:nth-child(1) > td:nth-child(2)').text.split('세대')[0]
                apt_company = web.find_elements(By.CLASS_NAME, 'info_table_item')[3].find_element(By.CLASS_NAME, 'table_td').text
                try:
                    web.find_element(By.XPATH,'//*[@id="detailContents1"]/div[2]/div[2]/div/div[2]/button')
                    web.find_element(By.XPATH,'//*[@id="detailContents1"]/div[2]/div[2]/div/div[2]/button').click()
                except:
                    pass
                for n in range(len(web.find_elements(By.CLASS_NAME,'detail_sorting_inner')[0].find_elements(By.CLASS_NAME,'detail_sorting_tab'))):
                    web.find_element(By.XPATH,'//*[@id="tab' + str(n) + '"]').click()
                    web.implicitly_wait(3)
                    size = web.find_element(By.XPATH,'/html/body/div[2]/div/section/div[2]/div[2]/div/div[2]/div[3]/div[2]/div[3]/table/tbody/tr[1]/td').text.split('㎡')[1][1:]
                    room = web.find_element(By.XPATH,'/html/body/div[2]/div/section/div[2]/div[2]/div/div[2]/div[3]/div[2]/div[3]/table/tbody/tr[2]/td').text.split('/')[0][:1]
                    bath = web.find_element(By.XPATH,'/html/body/div[2]/div/section/div[2]/div[2]/div/div[2]/div[3]/div[2]/div[3]/table/tbody/tr[2]/td').text.split('/')[1][:1]

                    apt_lst2.append({'apt':apt,'zibun':zibun, 'new_zibun':zibun_nm, 'new_apt':apt_nm, 'road':road, 'new_road':road_nm, 'latitude':lat, 'longitude':long, 'company':apt_company, 'home_cnt':home, 'home_size':size, 'room_cnt':room, 'bath_cnt':bath, 'bigo':bigo})
                        
        except:
            try:
                web.find_element(By.CSS_SELECTOR, '#summaryInfo')
                web.find_element(By.XPATH,'//*[@id="summaryInfo"]/div[2]/div[2]/button[1]').click()
                web.implicitly_wait(5)
                zibun_nm = web.find_element(By.CSS_SELECTOR, '#detailContents1 > div.detail_box--complex > table > tbody > tr:nth-child(7) > td > p:nth-child(1)').text.split('시 ')[-1]
                road_nm = web.find_elements(By.CLASS_NAME,'address')[-1].text.split('구 ')[-1]
                apt_nm = web.find_element(By.ID,'complexTitle').text
                lat = web.current_url.split('=')[1].split(',')[0]
                long = web.current_url.split('=')[1].split(',')[1]
                home = web.find_element(By.CSS_SELECTOR,'#detailContents1 > div.detail_box--complex > table > tbody > tr:nth-child(1) > td:nth-child(2)').text.split('세대')[0]
                apt_company = web.find_elements(By.CLASS_NAME, 'info_table_item')[3].find_element(By.CLASS_NAME, 'table_td').text
                try:
                    web.find_element(By.XPATH,'//*[@id="detailContents1"]/div[2]/div[2]/div/div[2]/button')
                    web.find_element(By.XPATH,'//*[@id="detailContents1"]/div[2]/div[2]/div/div[2]/button').click()
                except:
                    pass
                for n in range(len(web.find_elements(By.CLASS_NAME,'detail_sorting_inner')[0].find_elements(By.CLASS_NAME,'detail_sorting_tab'))):
                    web.find_element(By.XPATH,'//*[@id="tab' + str(n) + '"]').click()
                    web.implicitly_wait(3)
                    size = web.find_element(By.CSS_SELECTOR,'#tabpanel > table > tbody > tr:nth-child(1) > td').text.split('㎡')[1][1:]
                    room = web.find_element(By.CSS_SELECTOR,'#tabpanel > table > tbody > tr:nth-child(2) > td').text.split('/')[0][:1]
                    bath = web.find_element(By.CSS_SELECTOR,'#tabpanel > table > tbody > tr:nth-child(2) > td').text.split('/')[1][:1]
                    
                    apt_lst2.append({'apt':apt,'zibun':zibun, 'new_zibun':zibun_nm, 'new_apt':apt_nm, 'road':road, 'new_road':road_nm, 'latitude':lat, 'longitude':long, 'company':apt_company, 'home_cnt':home, 'home_size':size, 'room_cnt':room, 'bath_cnt':bath, 'bigo':bigo})
                
            except:
                    web.get('https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=' + sch_road)
                    web.implicitly_wait(5)
                    try:
                        web.find_element(By.CSS_SELECTOR, '#loc-main-section-root > section > div > div._3ttP1 > div > div._2p8Of > div._2ZgN5 > div._1oJkn > span:nth-child(2)')
                        apt_nm = web.find_element(By.CSS_SELECTOR, '#loc-main-section-root > section > div > div._3ttP1 > div > div._2p8Of > div._2ZgN5 > div._1oJkn > span:nth-child(2)').text.split('지번')[0]
                    except:
                        apt_nm = ''
                        bigo='아파트없음'
                        print('error', a1, a, apt)
                        apt_lst2.append({'apt':apt,'zibun':zibun, 'new_zibun':zibun_nm, 'new_apt':apt_nm, 'road':road, 'new_road':road_nm, 'latitude':lat, 'longitude':long, 'company':apt_company, 'home_cnt':home, 'home_size':size, 'room_cnt':room, 'bath_cnt':bath, 'bigo':bigo})
 
                    if apt_nm != '':
                        web.get('https://new.land.naver.com/complexes/667?ms=37.483998,127.056011,17&a=APT:ABYG:JGC&e=RETAIL')
                        web.implicitly_wait(5)
                        web.find_element(By.ID, 'search_input').clear()
                        web.find_element(By.ID, 'search_input').send_keys(apt_nm)
                        web.find_element(By.XPATH,'//*[@id="wrap"]/header/div[1]/form/fieldset/button[1]').click()
                        web.implicitly_wait(5)
                        try:
                            web.find_element(By.CSS_SELECTOR,'#ct > div.map_wrap > div.search_panel > div.list_fixed > div')
                            try:
                                web.find_element(By.XPATH, '//*[@id="ct"]/div[2]/div[2]/div[2]/div/div/div/div/i')
                                print('error', a1,  a, apt)
                                apt_lst2.append({'apt':apt,'zibun':zibun, 'new_zibun':zibun_nm, 'new_apt':apt_nm, 'road':road, 'new_road':road_nm, 'latitude':lat, 'longitude':long, 'company':apt_company, 'home_cnt':home, 'home_size':size, 'room_cnt':room, 'bath_cnt':bath, 'bigo':bigo})
                            except:
                                num_lst = []
                                for i in range(len(web.find_element(By.CSS_SELECTOR, '#ct > div.map_wrap > div.search_panel > div.list_contents > div').find_elements(By.CLASS_NAME, 'item'))):
                                    if (len(web.find_element(By.CSS_SELECTOR,'#ct > div.map_wrap > div.search_panel > div.list_contents > div > div > div:nth-child('+str(i+1)+') > div > a').find_elements(By.CSS_SELECTOR,'*')) > 1):
                                        if (web.find_element(By.CSS_SELECTOR, '#ct > div.map_wrap > div.search_panel > div.list_contents > div > div > div:nth-child('+str(i+1)+') > div > a > div.info_area > strong').text == '아파트') & (web.find_element(By.CSS_SELECTOR, '#ct > div.map_wrap > div.search_panel > div.list_contents > div > div > div:nth-child('+str(i+1)+') > div > a > div.address').text.endswith(apart_df[(apart_df.동_단지.str.startswith(apt_df.apt[a])) & (apart_df.도로명 == apt_df.road[a])]['시군구'].unique()[0].split('특별시 ')[-1])):
                                            num_lst.append(i)
                                if len(num_lst)==1:
                                    max_i = num_lst[0]
                                elif len(num_lst)>1:
                                    for i in num_lst:
                                        web.find_element(By.CSS_SELECTOR, '#ct > div.map_wrap > div.search_panel > div.list_contents > div > div > div:nth-child('+str(i+1)+')').click()
                                        web.implicitly_wait(5)
                                        web.find_element(By.XPATH,'//*[@id="summaryInfo"]/div[2]/div[2]/button[1]').click()
                                        web.implicitly_wait(5)
                                        if (web.find_elements(By.CSS_SELECTOR,'#detailContents1 > div.detail_box--complex > table > tbody > tr:nth-child(7) > td > p:nth-child(2)')[-1].text.split('구 ')[-1].split(' ')[0].endswith(road.split(' ')[0])) and (web.find_element(By.CSS_SELECTOR, '#detailContents1 > div.detail_box--complex > table > tbody > tr:nth-child(7) > td > p:nth-child(1)').text.split('-')[0].endswith(sch.split('특별시 ')[-1].split('-')[0])):
                                            zibun_nm1 = web.find_element(By.CSS_SELECTOR, '#detailContents1 > div.detail_box--complex > table > tbody > tr:nth-child(7) > td > p:nth-child(1)').text.split('시 ')[-1]
                                            road_nm1 = web.find_elements(By.CLASS_NAME,'address')[-1].text.split('구 ')[-1]
                                            if max(SequenceMatcher(None,zibun,zibun_nm1).ratio(), SequenceMatcher(None,road,road_nm1).ratio()) > rt:
                                                rt = max(SequenceMatcher(None,zibun,zibun_nm1).ratio(), SequenceMatcher(None,road,road_nm1).ratio())
                                                max_i = i
                                        web.find_element(By.ID, 'search_input').send_keys(apt_nm)
                                        web.find_element(By.XPATH,'//*[@id="wrap"]/header/div[1]/form/fieldset/button[1]').click()
                                        web.implicitly_wait(5)
                                web.find_element(By.CSS_SELECTOR, '#ct > div.map_wrap > div.search_panel > div.list_contents > div > div > div:nth-child('+str(max_i+1)+')').click()
                                web.implicitly_wait(5)
                                web.find_element(By.XPATH,'//*[@id="summaryInfo"]/div[2]/div[2]/button[1]').click()
                                web.implicitly_wait(5)
                                zibun_nm = web.find_element(By.CSS_SELECTOR, '#detailContents1 > div.detail_box--complex > table > tbody > tr:nth-child(7) > td > p:nth-child(1)').text.split('시 ')[-1]
                                road_nm = web.find_elements(By.CLASS_NAME,'address')[-1].text.split('구 ')[-1]
                                apt_nm = web.find_element(By.ID,'complexTitle').text
                                lat = web.current_url.split('=')[1].split(',')[0]
                                long = web.current_url.split('=')[1].split(',')[1]
                                home = web.find_element(By.CSS_SELECTOR,'#detailContents1 > div.detail_box--complex > table > tbody > tr:nth-child(1) > td:nth-child(2)').text.split('세대')[0]
                                try:
                                    web.find_element(By.XPATH,'//*[@id="detailContents1"]/div[2]/div[2]/div/div[2]/button')
                                    web.find_element(By.XPATH,'//*[@id="detailContents1"]/div[2]/div[2]/div/div[2]/button').click()
                                except:
                                    pass
                                for n in range(len(web.find_elements(By.CLASS_NAME,'detail_sorting_inner')[0].find_elements(By.CLASS_NAME,'detail_sorting_tab'))):
                                    web.find_element(By.XPATH,'//*[@id="tab' + str(n) + '"]').click()
                                    web.implicitly_wait(3)
                                    size = web.find_element(By.XPATH,'/html/body/div[2]/div/section/div[2]/div[2]/div/div[2]/div[3]/div[2]/div[3]/table/tbody/tr[1]/td').text.split('㎡')[1][1:]
                                    room = web.find_element(By.XPATH,'/html/body/div[2]/div/section/div[2]/div[2]/div/div[2]/div[3]/div[2]/div[3]/table/tbody/tr[2]/td').text.split('/')[0][:1]
                                    bath = web.find_element(By.XPATH,'/html/body/div[2]/div/section/div[2]/div[2]/div/div[2]/div[3]/div[2]/div[3]/table/tbody/tr[2]/td').text.split('/')[1][:1]

                                    apt_lst2.append({'apt':apt,'zibun':zibun, 'new_zibun':zibun_nm, 'new_apt':apt_nm, 'road':road, 'new_road':road_nm, 'latitude':lat, 'longitude':long, 'company':apt_company, 'home_cnt':home, 'home_size':size, 'room_cnt':room, 'bath_cnt':bath, 'bigo':bigo})
                        except:
                            print('error', a1, a, apt)
                            bigo = '아파트없음'
                            apt_lst2.append({'apt':apt,'zibun':zibun, 'new_zibun':zibun_nm, 'new_apt':apt_nm, 'road':road, 'new_road':road_nm, 'latitude':lat, 'longitude':long, 'company':apt_company, 'home_cnt':home, 'home_size':size, 'room_cnt':room, 'bath_cnt':bath, 'bigo':bigo})
    else:
        print('error', a1, a, apt)
        bigo = '아파트없음'
        apt_lst2.append({'apt':apt,'zibun':zibun, 'new_zibun':zibun_nm, 'new_apt':apt_nm, 'road':road, 'new_road':road_nm, 'latitude':lat, 'longitude':long, 'company':apt_company, 'home_cnt':home, 'home_size':size, 'room_cnt':room, 'bath_cnt':bath, 'bigo':bigo})
                 


In [ ]:
# 리스트 확인
apt_lst2

In [ ]:
# 데이터프레임 확인
pd.DataFrame(apt_lst2)